<a href="https://colab.research.google.com/github/roshantac/EVA4/blob/master/Assignment15/A/Depthmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13656095251030506674, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6085412704117920303
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2638685563396963352
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7304675328
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4645847619114219135
 physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"]

In [2]:
!git clone https://github.com/roshantac/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 5), reused 47 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [3]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-05-07 16:29:09--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.56.227
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.56.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  10.3MB/s    in 18s     

2020-05-07 16:29:28 (9.16 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp '/content/gdrive/My Drive/dataSet2.zip' '/content/DenseDepth/'

In [5]:
%cd DenseDepth

/content/DenseDepth


In [0]:
mkdir 'Data'

In [0]:
destDataDir = '/content/DenseDepth/Data/'
path_to_zip_file = '/content/DenseDepth/dataSet2.zip'
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(destDataDir)

In [0]:
#!rm -rf Data/DepthMAp/

In [0]:
import skimage
from skimage.transform import resize
import os
import shutil
import numpy as np
from PIL import Image
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from layers import BilinearUpSampling2D
from utils import predict, load_images, display_images
from matplotlib import pyplot as plt


try:
  os.mkdir("content/DenseDepth/Data/DepthMap")
except:
  print("Either folder already exists or tree is not exist")

srcpath = "/content/DenseDepth/Data/Output/" #"/content/DenseDepth/Data/Output/fgbg/"
destpath = "/content/DenseDepth/Data/DepthMap/" #"/content/DenseDepth/Data/DepthMap/"
prrMdl = 'nyu.h5'
# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}
print('Loading model...')
model = load_model(prrMdl, custom_objects=custom_objects, compile=False)
print('loaded ...')
for idx,d in enumerate(os.listdir(srcpath)):
  loaded_images = []
  imgsrc = srcpath+ d
  
  print("imgsrc",imgsrc)
  img = np.clip(np.asarray((Image.open(imgsrc)).resize((640,480),Image.BILINEAR), dtype=float) / 255, 0, 1)
  #loaded_images.append(img)
  #img =np.stack(loaded_images, axis=0)
  outputs = predict(model, img)
#for i in range(outputs.shape[0]):
  print(outputs.shape)
  rescaled = outputs[0][:,:,0]
  print("rscaled ",rescaled.shape ,"  out: ",outputs[0].shape)
  #rescaled = rescaled - np.min(rescaled)
  #rescaled = rescaled / np.max(rescaled)
  im = Image.fromarray(255-np.uint8(rescaled*255))
  im.save(destpath + d, optimize=True, quality=50)
  print(idx)

In [9]:
print("he")

he
